# In this notebook, we have tried to establish a relation between certain parameters from survery and tried to predict the mental health of a adult

## Import

In [1]:
# Author: Jayant Rane
import numpy as np
import pandas as pd
import math
print(np.__version__)
print(pd.__version__)
#!pip install numpy==1.19.5 --user
#Updating to pandas>=1.0.5 should solve it. Supposedly also pandas==0.25.3 works. If you are stuck with pandas 0.24.x you may not be able to usse numpy 1.20.x though, unfortunaly. In that case, use numpy<1.20,

pd.set_option('display.max_columns', 60)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(font_scale = 2)

from sklearn.model_selection import train_test_split

from IPython.core.display import HTML

import matplotlib.dates as mdates

import os

from sklearn import preprocessing,metrics 
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from sklearn import linear_model,svm
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

1.13.3
0.19.2


## Adult Data April 2020

In [2]:
dfapr20 = pd.read_csv("Data/ProlificAcademic/updated_data/April/CRISIS_Adult_April_2020.csv")
dfapr20data = pd.read_csv("Data/ProlificAcademic/April 2020/Dictionaries/CRISIS_Adult_April_Dictionary.csv")

dfapr21 = pd.read_csv("Data/ProlificAcademic/updated_data/April_21/CRISIS_Adult_April_2021.csv")
dfapr21data = pd.read_csv("Data/ProlificAcademic/April 2021/Dictionaries/CRISISPAAdultApril2021_DataDictionary_2021-11-01.csv")

traindata = dfapr20.copy()
print(traindata.head())
print(traindata.info())
print(traindata.describe())
#traindata.head(10)
traindata.shape

         ID           timestamp1  country   age  sex sex_other  \
0  00136f50  2020-04-09 00:13:00    187.0  18.0  1.0       NaN   
1  003db2d0  2020-04-09 16:39:00    187.0  34.0  1.0       NaN   
2  005a1e3e  2020-04-09 17:10:00    187.0  24.0  2.0       NaN   
3  005ef0c4  2020-04-09 16:18:00    187.0  51.0  1.0       NaN   
4  00601150  2020-04-09 16:04:00    187.0  38.0  2.0       NaN   

   raceethnicity___1  raceethnicity___2  raceethnicity___3  raceethnicity___4  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  0   
2                  0                  0                  0                  0   
3                  1                  0                  0                  0   
4                  0                  0                  0                  0   

   raceethnicity___5  raceethnicity___6  raceethnicity___7  raceethnicity___8  \
0                  0               

(2868, 244)

### Data Describe

In [3]:
# finding common persons in both dataset
dfapr21['ID'].value_counts(sort=True)
print(dfapr20.shape)
print(dfapr21.shape)
len(list(set(dfapr21.ID) & set(dfapr21.ID)))

(2868, 244)
(2125, 286)


2125

In [4]:
# race could be 
apr20racestr = "1, England, Ireland, Scotland or Wales | 2, Australia - not of Aboriginal or Torres Strait Islander descent | 3, Australia - of Aboriginal or Torres Strait Islander descent | 4, New Zealand - not of Maori descent | 5, New Zealand - of Maori descent | 6, Northern Europe including Sweden, Norway, Finland and surrounding countries | 7, Western Europe including France, Germany, the Netherlands and surrounding countries | 8, Eastern Europe including Russia, Poland, Hungary and surrounding countries | 9, Southern Europe including Italy, Greece, Spain, Portugal and surrounding countries | 10, Middle East including Lebanon, Turkey and surrounding countries | 11, Eastern Asia including China, Japan, South Korea, North Korea, Taiwan and Hong Kong | 12, South-East Asia including Thailand, Malaysia, Indonesia, Singapore and surrounding countries | 13, South Asia including India, Pakistan, Sri Lanka and surrounding countries | 14, Polynesia, Micronesia or Melanesia including Tonga, Fiji, Papua New Guinea and surrounding countries | 15, Africa | 16, North America - not of First Nations, Native American, Inuit or Métis descent | 17, North America - of First Nations, Native American, Inuit or Métis descent | 18, Caribbean, Central or South America | 19, Don't know | 20, Other"
delim = '|'
racelist = apr20racestr.split(delim)
racelist = [x.strip(' ') for x in racelist]
racemaps = { i : racelist[i-1].split(',')[1].strip(' ') for i in range(1, len(racelist)+1 ) } 
racemaps

{1: 'England',
 2: 'Australia - not of Aboriginal or Torres Strait Islander descent',
 3: 'Australia - of Aboriginal or Torres Strait Islander descent',
 4: 'New Zealand - not of Maori descent',
 5: 'New Zealand - of Maori descent',
 6: 'Northern Europe including Sweden',
 7: 'Western Europe including France',
 8: 'Eastern Europe including Russia',
 9: 'Southern Europe including Italy',
 10: 'Middle East including Lebanon',
 11: 'Eastern Asia including China',
 12: 'South-East Asia including Thailand',
 13: 'South Asia including India',
 14: 'Polynesia',
 15: 'Africa',
 16: 'North America - not of First Nations',
 17: 'North America - of First Nations',
 18: 'Caribbean',
 19: "Don't know",
 20: 'Other'}

In [5]:
dfapr20new = pd.DataFrame()
def getrace(row, dfapr20new):
    raceeth = 'raceethnicity___'
    for i in range(1, 21):
        if (row[raceeth+str(i)] != 0):
            ser = pd.Series([row['ID'], racemaps[i]], index=['ID','race'])
            print(row['ID'] + " " + racemaps[i])
            dfapr20new = dfapr20new.append(ser,ignore_index=True)
    return dfapr20new

dfapr20new = getrace(dfapr20.iloc[2], dfapr20new)
dfapr20new

for i in range(len(dfapr20)):
    dfapr20new = getrace(dfapr20.iloc[i], dfapr20new)
dfapr20new  

005a1e3e Middle East including Lebanon
003db2d0 Eastern Europe including Russia
005a1e3e Middle East including Lebanon
005ef0c4 England
005ef0c4 Western Europe including France
00601150 Western Europe including France
00601150 Eastern Europe including Russia
00769460 England
0082e6f9 England
00a41e77 Eastern Europe including Russia
00babb90 Southern Europe including Italy
00bb1f2d Western Europe including France
00db60f1 England
01297d76 England
01297d76 North America - not of First Nations
01559bfb Eastern Europe including Russia
015a3ade Eastern Asia including China
01621335 Southern Europe including Italy
016d8f74 England
016d8f74 New Zealand - not of Maori descent
01808e74 Don't know
0180ea1d England
01a97318 England
01c706a7 Southern Europe including Italy
01c82406 Western Europe including France
01c82406 Middle East including Lebanon
0206c601 England
02650a88 North America - of First Nations
027eaa15 England
027eaa15 Western Europe including France
027eaa15 Eastern Europe includi

,ID,race
0,005a1e3e,Middle East including Lebanon
1,003db2d0,Eastern Europe including Russia
2,005a1e3e,Middle East including Lebanon
3,005ef0c4,England
4,005ef0c4,Western Europe including France
5,00601150,Western Europe including France
6,00601150,Eastern Europe including Russia
7,00769460,England
8,0082e6f9,England
9,00a41e77,Eastern Europe including Russia


### Data Cleaning

In [6]:
# Here we are checking for every parameter if its values are are not skewed and if we can use that parameters
traindata['sample_country'].value_counts()

UK    1440
US    1428
Name: sample_country, dtype: int64

In [7]:
# Categorical columns
cat_cols = ['sex', 'location', 'education','empstat','racecat1']

In [8]:
# These columns have good value, we will be using cols which we these words
useful_cols = ['raceethnicity___', 'working___','military','householdnumber','householdcomp___','roomsinhouse','insurance',
              'govassist', 'physicalhealth','healthconditions___','symptoms___','exposed___','infected',
               'worried','prior','threemonths','disruptedsupports___','twoweeks','sample_country','empstat'
               ,'racecat1','fam_','age_bin']
#'mentalhealth'          'school___'

In [9]:
# this cols are droped as info is either duplicate or non-usable directly
not_useful_cols = ['ID', 'timestamp1', 'sex_other', 'country', 'age', 'occupation','tall']

In [10]:
# creating the pandas df with useful cols
df = traindata.loc[:,  (traindata.columns.str.startswith('ID')) ]
for col in useful_cols:
    df1 = traindata.loc[:,  (traindata.columns.str.startswith(col)) ]
    df = pd.concat([df, df1], axis=1)


In [11]:
# Update age bin
df['age_bin'] = df['age_bin'].map({'30_under':1,  
      '30-49':2,  '50_over':3 })
# Update country
df['sample_country'] = df['sample_country'].map({'US':1,  
      'UK':2 })

In [12]:
# cleaning na values
print(df.isnull().any() == True)
df.select_dtypes(include=['object'])
df = df.drop(['ID'], axis=1)
df = df.fillna(df.mean())
#df.replace({-np.inf: -1_000_000, np.inf: 1_000_000}, inplace=True)

ID                        False
raceethnicity___1         False
raceethnicity___2         False
raceethnicity___3         False
raceethnicity___4         False
raceethnicity___5         False
raceethnicity___6         False
raceethnicity___7         False
raceethnicity___8         False
raceethnicity___9         False
raceethnicity___10        False
raceethnicity___11        False
raceethnicity___12        False
raceethnicity___13        False
raceethnicity___14        False
raceethnicity___15        False
raceethnicity___16        False
raceethnicity___17        False
raceethnicity___18        False
raceethnicity___19        False
raceethnicity___20        False
working___1               False
working___2               False
working___3               False
working___4               False
working___5               False
working___6               False
working___7               False
working___8               False
military                   True
                          ...  
disrupte

In [13]:
# proving that no nan or inf numbers are present
print(np.any(np.isnan(df)))

print(np.all(np.isfinite(df)))

print(np.any(np.isinf(df)))
for col in df.columns:
    print(col + " " + str(np.where(np.isinf(df.loc[:, col]))))

False
True
False
raceethnicity___1 (array([], dtype=int64),)
raceethnicity___2 (array([], dtype=int64),)
raceethnicity___3 (array([], dtype=int64),)
raceethnicity___4 (array([], dtype=int64),)
raceethnicity___5 (array([], dtype=int64),)
raceethnicity___6 (array([], dtype=int64),)
raceethnicity___7 (array([], dtype=int64),)
raceethnicity___8 (array([], dtype=int64),)
raceethnicity___9 (array([], dtype=int64),)
raceethnicity___10 (array([], dtype=int64),)
raceethnicity___11 (array([], dtype=int64),)
raceethnicity___12 (array([], dtype=int64),)
raceethnicity___13 (array([], dtype=int64),)
raceethnicity___14 (array([], dtype=int64),)
raceethnicity___15 (array([], dtype=int64),)
raceethnicity___16 (array([], dtype=int64),)
raceethnicity___17 (array([], dtype=int64),)
raceethnicity___18 (array([], dtype=int64),)
raceethnicity___19 (array([], dtype=int64),)
raceethnicity___20 (array([], dtype=int64),)
working___1 (array([], dtype=int64),)
working___2 (array([], dtype=int64),)
working___3 (arr

### Feature Processing

In [14]:
# Tring to see if using standard scaling will provide better accuracy
ss = preprocessing.StandardScaler()
fields = df.columns
data_clean = pd.DataFrame(ss.fit_transform(df[fields]), columns = fields)

### Train test split




In [15]:
# creating train test data,and dropping rows for which target variable value is absent
X = data_clean.copy()
y_train = traindata['mentalhealth'].copy()
rows = np.where(np.isnan(y_train))
print(rows[0].tolist())
X.drop(rows[0].tolist(), inplace=True)
y_train.drop(rows[0].tolist(), inplace=True)

[201, 1165, 1520, 1574, 1690, 2130, 2676]


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_train, test_size=0.20, random_state=42)

In [17]:
X_train

,raceethnicity___1,raceethnicity___2,raceethnicity___3,raceethnicity___4,raceethnicity___5,raceethnicity___6,raceethnicity___7,raceethnicity___8,raceethnicity___9,raceethnicity___10,raceethnicity___11,raceethnicity___12,raceethnicity___13,raceethnicity___14,raceethnicity___15,raceethnicity___16,raceethnicity___17,raceethnicity___18,raceethnicity___19,raceethnicity___20,working___1,working___2,working___3,working___4,working___5,working___6,working___7,working___8,military,householdnumber,...,disruptedsupports___1,disruptedsupports___2,disruptedsupports___3,disruptedsupports___4,disruptedsupports___5,disruptedsupports___6,disruptedsupports___7,disruptedsupports___8,disruptedsupports___9,disruptedsupports___10,disruptedsupports___11,disruptedsupports___12,disruptedsupports___13,twoweeksalcohol,twoweeksvaping,twoweekstobacco,twoweeksmarijuana,twoweeksopiates,twoweeksother,twoweeksleeping,sample_country,empstatus,empstat,racecat1,fam_esntl_wrkr,fam_impact_any,fam_impact_fin,fam_impact_hlth,fam_dx,age_bin
2028,0.882698,-0.049464,-0.026417,-0.052889,-0.026417,-0.229079,-0.464510,-0.376457,-0.328925,-0.166098,-0.239838,-0.186097,-0.204383,-0.04179,-0.227388,-0.235746,-0.216137,-0.178958,-0.146176,-0.026417,-1.016170,-0.237389,3.548580,3.187402,-0.278182,-0.295972,-0.172634,-0.447775,-0.199588,0.518744,...,-0.236568,-0.327622,-0.258775,-0.394360,-0.393769,-0.263346,-0.246274,-0.184082,-0.254918,-0.154661,1.663949,-0.341793,-0.280359,1.188942,-0.310538,-0.361237,-0.309488,-0.131441,-0.136518,-0.298074,0.995825,-0.206718,-0.013011,-6.480659e-01,2.703937,-0.830420,-0.653225,-5.172728e-01,-0.242678,-1.151256
652,0.882698,-0.049464,-0.026417,-0.052889,-0.026417,-0.229079,-0.464510,-0.376457,-0.328925,-0.166098,-0.239838,-0.186097,-0.204383,-0.04179,-0.227388,-0.235746,-0.216137,-0.178958,-0.146176,-0.026417,-1.016170,4.212493,-0.281803,-0.313735,-0.278182,-0.295972,-0.172634,-0.447775,-0.199588,0.518744,...,-0.236568,-0.327622,-0.258775,-0.394360,-0.393769,-0.263346,-0.246274,-0.184082,-0.254918,-0.154661,-0.600980,-0.341793,-0.280359,1.612220,-0.310538,-0.361237,-0.309488,-0.131441,-0.136518,-0.298074,0.995825,1.021021,-0.013011,-6.480659e-01,-0.642948,-0.830420,-0.653225,-5.172728e-01,-0.242678,1.436475
2228,0.882698,-0.049464,-0.026417,-0.052889,-0.026417,-0.229079,-0.464510,-0.376457,-0.328925,-0.166098,-0.239838,-0.186097,-0.204383,-0.04179,-0.227388,-0.235746,-0.216137,-0.178958,-0.146176,-0.026417,0.984088,-0.237389,-0.281803,-0.313735,-0.278182,-0.295972,-0.172634,-0.447775,-0.199588,-0.856216,...,-0.236568,-0.327622,-0.258775,-0.394360,-0.393769,-0.263346,-0.246274,-0.184082,-0.254918,-0.154661,-0.600980,-0.341793,-0.280359,1.188942,-0.310538,-0.361237,-0.309488,-0.131441,-0.136518,-0.298074,0.995825,-0.820587,-0.890661,-6.480659e-01,-0.642948,-0.830420,-0.653225,-5.172728e-01,-0.242678,0.142610
2051,-1.132890,-0.049464,-0.026417,-0.052889,-0.026417,-0.229079,-0.464510,-0.376457,-0.328925,-0.166098,-0.239838,-0.186097,-0.204383,-0.04179,-0.227388,4.241860,-0.216137,-0.178958,-0.146176,-0.026417,0.984088,-0.237389,-0.281803,-0.313735,-0.278182,-0.295972,-0.172634,-0.447775,-0.199588,-1.543696,...,-0.236568,-0.327622,-0.258775,-0.394360,-0.393769,-0.263346,-0.246274,-0.184082,-0.254918,-0.154661,1.663949,-0.341793,-0.280359,-0.927449,-0.310538,-0.361237,-0.309488,-0.131441,-0.136518,-0.298074,-1.004193,-0.820587,-0.890661,-1.875936e-01,2.703937,-0.830420,-0.653225,-5.172728e-01,-0.242678,1.436475
671,0.882698,-0.049464,-0.026417,-0.052889,-0.026417,-0.229079,-0.464510,-0.376457,-0.328925,-0.166098,-0.239838,-0.186097,-0.204383,-0.04179,-0.227388,-0.235746,-0.216137,-0.178958,-0.146176,-0.026417,0.984088,4.212493,-0.281803,-0.313735,-0.278182,-0.295972,-0.172634,-0.447775,-0.199588,0.518744,...,-0.236568,-0.327622,-0.258775,-0.394360,-0.393769,-0.263346,-0.246274,-0.184082,-0.254918,-0.154661,-0.600980,-0.341793,-0.280359,1.188942,-0.310538,-0.361237,-0.309488,-0.131441,-0.136518,-0.298074,0.995825,-0.820587,-0.890661,-6.480659e-01,1

### Stability selection

In [18]:
# ranks the features using stats
ranks = {}
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [19]:
# Using Linear Regression
lr = LinearRegression(normalize=True)
lr.fit(X_train,y_train)
ranks["LinReg"] = ranking(np.abs(lr.coef_), X_train.columns)

In [20]:
# applying the model
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)


y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Real Values':y_test, 'Predicted Values':y_pred})
df

,Predicted Values,Real Values
1591,2.399346,2.0
2209,1.298569,1.0
773,2.166466,3.0
1737,2.283560,1.0
388,2.421073,1.0
408,2.929587,3.0
2302,2.268842,3.0
2458,1.647814,1.0
2644,4.125918,5.0
2764,1.685189,2.0


## Evaluate models

In [21]:
# MAE  an arithmetic average of the absolute errors
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, y_pred))

# this is not accuracy but mean of difference between real and prediction
# it signifies if real is 3 hence pred could be (3 +- MAE)

0.87237485253


In [ ]:
!pip3 freeze > requirements.txt